In [102]:
import utils as my
import pandas as pd
import pubchempy as pbp
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
import torch
import numpy as np
from torch import nn
from tqdm import tqdm
import plotly

In [103]:
data = pd.read_csv('../data.csv')
smiles = data['smiles']
CI_raw = data['A2780 CS']
smiles_nonan = []
CI = []
for i in range(len(smiles)):
    if not pd.isna(smiles[i]) and not pd.isna(CI_raw[i]):
        smiles_nonan.append(smiles[i])
        CI.append(my.strtonp(CI_raw[i]))
CI.pop(272)
smiles_nonan.pop(272)
CI = np.asarray(CI)
CI = np.log(CI)

In [104]:
print(CI[271], np.argmax(CI[:, 1]))

[0.11897743 4.25148853 2.20185098] 271


In [105]:
fingerprints = []
for i in smiles_nonan:
    molecular = Chem.MolFromSmiles(i)
    fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(molecular, 6, nBits=1024)
    fingerprint = list(fingerprint)
    fingerprints.append(fingerprint)


[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerator
[19:52:08] DEPRECATION WARNING: please use MorganGenerat

In [106]:
features = torch.tensor(fingerprints, dtype=torch.float)
labels = torch.tensor(CI[:,1], dtype=torch.float).unsqueeze(1)
# data_processed = pd.DataFrame({'features': features, 'labels': labels})
print(labels.shape)

torch.Size([340, 1])


In [107]:
# import torch
# from torchvision import transforms

# # 定义标准化变换
# transform = transforms.Lambda(lambda x: (x - torch.mean(x)) / torch.std(x))

# # 应用变换
# labels = transform(labels)

In [115]:
from torch.utils import data
def load_array(array, batchsize, is_train=True):
    dataset = data.TensorDataset(*array)
    return data.DataLoader(dataset, batch_size=batchsize, shuffle=is_train)
dataloader = load_array((features, labels), 128)
net = nn.Sequential(nn.Linear(1024, 256),
                    nn.ReLU(),
                    nn.Linear(256, 1))
trainer = torch.optim.SGD(net.parameters(), lr=0.0003)
loss = nn.MSELoss()
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, 0, std=1)

net.apply(init_weights)
y_ = net(features)
l = loss(y_, labels)
l.backward()
print(net[0].weight.grad)
# num_epochs = 100
# for epoch in tqdm(range(num_epochs)):
#     for X, y in dataloader:
#         l = loss(net(X) ,y)
#         trainer.zero_grad()
#         l.backward()
#         trainer.step()
#     l = loss(net(features), labels)
# print(f'epoch {epoch + 1}, loss {l:f}')
# l = loss(net(features), labels)
# print(f'epoch {epoch + 1}, loss {l:f}')
# import plotly.graph_objects as go
# net.train(False)
# y_ = net(features)
# x = y_.flatten().detach()
# y = labels.flatten().detach()
# fig = go.Figure()
# # fig.add_trace(go.Scatter(x = features.flatten(), y=labels.flatten(), mode='markers'))
# # fig.add_trace(go.Scatter(x = features.flatten(), y= x))
# fig.add_trace(go.Scatter(x = x, y=y, mode='markers'))

# # 显示图表
# fig.show()

tensor([[-1.6659e-01, -4.6162e-01, -1.0526e-01,  ..., -6.1179e-02,
         -7.6832e-02, -9.2141e-02],
        [ 1.2039e+01,  3.9574e+01,  7.3676e+00,  ...,  2.3887e+00,
          6.0984e+00,  1.0006e+01],
        [ 1.1832e+00,  7.0382e+00,  1.1492e+00,  ...,  1.1837e+00,
          3.3093e-01,  2.0332e+00],
        ...,
        [ 8.8596e+00,  2.8431e+01,  5.7291e+00,  ...,  6.2459e+00,
          4.0612e+00,  8.2329e+00],
        [ 8.3617e+00,  2.8753e+01,  1.1575e+01,  ...,  5.0536e+00,
          3.4420e+00,  6.9223e+00],
        [ 1.6193e+01,  7.2131e+01,  1.9475e+01,  ...,  1.0950e+01,
          9.9497e+00,  2.2481e+01]])


In [109]:
net[0].weight.data

tensor([[ 0.3536, -0.1937,  0.8767,  ..., -0.1486, -0.5552,  0.3578],
        [-0.5870, -0.6922,  0.2027,  ...,  0.2946,  0.3991,  1.9718],
        [ 0.1054,  0.8598,  0.2579,  ..., -0.4022,  0.8294, -1.2035],
        ...,
        [-0.5595, -0.8846, -0.3785,  ...,  0.2600,  0.1396, -0.1911],
        [ 1.3548, -0.0467, -0.2475,  ..., -2.0946, -0.0498,  0.0341],
        [ 1.1536, -1.0649, -0.1055,  ..., -0.2052, -0.6490,  1.3233]])